In [125]:
import numpy as np 
import pandas as pd
import os, zipfile

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

from keras.utils import np_utils
from sklearn.utils import shuffle

In [3]:
path1 = 'C:/Users/Movetech/Desktop/Uni/AML/Kaggle/ocean_pirates/train_images' # path of folder of images    
path2 = 'C:/Users/Movetech/Desktop/Uni/AML/Kaggle/ocean_pirates/train_images_edited' # path of folder to save images

In [15]:
listing = os.listdir(path1)
df_labels = pd.read_csv('train_onelabel.csv')


print('Images and labels have the same length: ' + str(len(listing)==len(df_labels)))

Images and labels have the same length: True


In [30]:
# split df into images and labels
images = df_labels['image']
labels = df_labels['class']

In [31]:
im1 = Image.open('train_images/'+ df_labels.image[0]) # open one image to get size
im2 = Image.open('train_images/'+ df_labels.image[1]) # open one image to get size

In [87]:
# input image dimensions
img_rows, img_cols = 32, 32

# number of channels
img_channels = 1

# number of images
img_total = len(df_labels)

In [39]:
# change image dimension and save them
for file in listing:
    im = Image.open(path1 + '\\' + file)   
    img = im.resize((img_rows,img_cols))         
    img.save(path2 +'\\' +  file, "JPEG")

In [43]:
# just for testing

im1_edited = Image.open('train_images_edited/'+ df_labels.image[0]) # open one image to get size
im2_edited = Image.open('train_images_edited/'+ df_labels.image[1]) # open one image to get size

In [132]:
# create image matrix

image_matrix = np.empty((img_total, img_rows, img_cols))

for counter, image in enumerate(images):
    image_matrix[counter] = np.array(Image.open('train_images_edited/' + image))

In [133]:
# final preprocessing steps for images

image_matrix = image_matrix.astype('float32')
image_matrix /= 255

X_train = image_matrix

In [134]:
# preprocessing for labels

classes = len(df_labels.groupby('class'))
Y_train = np_utils.to_categorical(labels, classes)

In [135]:
# shuffle data
X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train, random_state=2)